In [1]:
class dummy():
    pass

In [21]:
args = dummy()

In [170]:
import sys
sys.path.append('..')
sys.path.append('../..')
import torch.optim as optim
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import os
import platform
import logging
from collections import OrderedDict
from models.ConvLSTM import EncoderForecaster, Encoder, Forecaster, ConvLSTMCell
from utils.io import save_network, save, create_results_folder, save_datasets_to_file
from utils.arg_extract import get_args
from utils.ExperimentBuilder import ExperimentBuilder
from utils.WaveDataset import create_datasets, transformVar, create_dataloaders
plt.ioff()
logging.basicConfig(format='%(message)s', level=logging.INFO)

# args, device = get_args()  # get arguments from command line
args.batch_size=6
args.experiment_name='ttt'
device = 'cpu'
# Data
if 'Darwin' in platform.system():
    base_folder = '/Users/stathis/Code/thesis/wave_propagation/'
    data_dir = base_folder
else:
    base_folder = '/home/s1680171/wave_propagation/'
    data_dir = '/disk/scratch/s1680171/wave_propagation/'

dirs = create_results_folder(base_folder=base_folder, experiment_name=experiment_name)

logging.info('Creating datasets')
train_dataset, val_dataset, test_dataset = create_datasets(os.path.join(data_dir, "Video_Data/"), 
                                                           transformVar, test_fraction=0.15,
                                                           validation_fraction=0.15)
# filename_data = os.path.join(dirs['pickles'], "all_data.pickle")
# save_datasets_to_file(train_dataset, val_dataset, test_dataset, filename_data)

train_dataloader, val_dataloader, test_dataloader = create_dataloaders(train_dataset, val_dataset, test_dataset, 
                                                                       batch_size=batch_size, num_workers=20)

Creating datasets


In [171]:
# for batch_num, batch_images in enumerate(train_dataloader):
#     print(batch_num, batch_images.size())

In [30]:
batch_images.size()

torch.Size([1, 100, 128, 128])

In [35]:
args.num_input_frames = 5
args.num_output_frames = 20
args.learning_rate = 10e-3

In [37]:
# Define encoder #
encoder_architecture = [
    [   # in_channels, out_channels, kernel_size, stride, padding
        OrderedDict({'conv1_leaky_1': [1, 8, 3, 2, 1]}),
        OrderedDict({'conv2_leaky_1': [64, 192, 3, 2, 1]}),
        OrderedDict({'conv3_leaky_1': [192, 192, 3, 2, 1]}),
    ],

    [
        ConvLSTMCell(input_channel=8, num_filter=64, b_h_w=(args.batch_size, 64, 64),
                     kernel_size=3, stride=1, padding=1, device=device, seq_len=args.num_input_frames),
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(args.batch_size, 32, 32),
                     kernel_size=3, stride=1, padding=1, device=device, seq_len=args.num_input_frames),
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(args.batch_size, 16, 16),
                     kernel_size=3, stride=1, padding=1, device=device, seq_len=args.num_input_frames),
    ]
]
forecaster_architecture = [
    [
        OrderedDict({'deconv1_leaky_1': [192, 192, 4, 2, 1]}),
        OrderedDict({'deconv2_leaky_1': [192, 64, 4, 2, 1]}),
        OrderedDict({
            'deconv3_leaky_1': [64, 8, 4, 2, 1],
            'conv3_leaky_2': [8, 8, 3, 1, 1],
            'conv3_3': [8, 1, 1, 1, 0]
        }),
    ],

    [
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(args.batch_size, 16, 16),
                     kernel_size=3, stride=1, padding=1, device=device, seq_len=args.num_output_frames),
        ConvLSTMCell(input_channel=192, num_filter=192, b_h_w=(args.batch_size, 32, 32),
                     kernel_size=3, stride=1, padding=1, device=device, seq_len=args.num_output_frames),
        ConvLSTMCell(input_channel=64, num_filter=64, b_h_w=(args.batch_size, 64, 64),
                     kernel_size=3, stride=1, padding=1, device=device, seq_len=args.num_output_frames),
    ]
]

encoder = Encoder(encoder_architecture[0], encoder_architecture[1]).to(device)
forecaster = Forecaster(forecaster_architecture[0], forecaster_architecture[1], args.num_output_frames).to(device)
model = EncoderForecaster(encoder, forecaster)

# optimizer = optim.Adam(model.parameters(), amsgrad=False, lr=args.learning_rate, weight_decay=args.weight_decay_coefficient)
# lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=7)


In [172]:
batch_ok = next(iter(train_dataloader))

In [174]:
batch_ok.min()

tensor(-1.)

In [54]:
out = model(batch_ok)

torch.Size([20, 6, 1, 128, 128])


In [55]:
from utils.helper_functions import convert_BSHW_to_SBCHW, convert_SBCHW_to_BSHW
cout = convert_SBCHW_to_BSHW(out)
cout.size()

torch.Size([6, 20, 128, 128])

In [72]:
out1 = model(batch_ok)

In [101]:
out2 = model(batch_images)

In [102]:
out2.size()

torch.Size([1, 20, 128, 128])

In [98]:
x = out1.clone()
print(x.size())
x = x.squeeze(2)
# x = x.squeeze()
print(x.size())
x = x.permute(1,0,2,3)
print(x.size())

torch.Size([20, 6, 1, 128, 128])
torch.Size([20, 6, 128, 128])
torch.Size([6, 20, 128, 128])


In [92]:
x = out2.clone()
print('original ', x.size())
x = x.squeeze(1)
print('squeeze ', x.size())

original  torch.Size([20, 1, 1, 128, 128])
squeeze  torch.Size([20, 1, 128, 128])


In [77]:
x = x.permute(1,0,2,3,)
print('permute ', x.size())

original  torch.Size([20, 1, 1, 128, 128])
squeeze  torch.Size([20, 128, 128])


RuntimeError: number of dims don't match in permute

In [103]:
model.encoder

Encoder(
  (stage1): Sequential(
    (conv1_leaky_1): Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (leaky_conv1_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn1): ConvLSTMCell(
    (_conv): Conv2d(72, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (stage2): Sequential(
    (conv2_leaky_1): Conv2d(64, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (leaky_conv2_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn2): ConvLSTMCell(
    (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (stage3): Sequential(
    (conv3_leaky_1): Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (leaky_conv3_leaky_1): LeakyReLU(negative_slope=0.2, inplace)
  )
  (rnn3): ConvLSTMCell(
    (_conv): Conv2d(384, 768, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
)

In [104]:
encoder = Encoder(encoder_architecture[0], encoder_architecture[1]).to(device)

[('conv1_leaky_1', Conv2d(1, 8, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))), ('leaky_conv1_leaky_1', LeakyReLU(negative_slope=0.2, inplace))]
[('conv2_leaky_1', Conv2d(64, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))), ('leaky_conv2_leaky_1', LeakyReLU(negative_slope=0.2, inplace))]
[('conv3_leaky_1', Conv2d(192, 192, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))), ('leaky_conv3_leaky_1', LeakyReLU(negative_slope=0.2, inplace))]


In [105]:
input = convert_BSHW_to_SBCHW(batch_images)
state = encoder(input)

In [108]:
batch_images.size()

torch.Size([1, 100, 128, 128])

In [107]:
input.size()

torch.Size([100, 1, 1, 128, 128])

In [123]:
print(state[0][0].size())
print(state[0][1].size())
print(state[1][0].size())
print(state[1][1].size())
print(state[2][0].size())
print(state[2][1].size())

torch.Size([1, 64, 64, 64])
torch.Size([1, 64, 64, 64])
torch.Size([1, 192, 32, 32])
torch.Size([1, 192, 32, 32])
torch.Size([1, 192, 16, 16])
torch.Size([1, 192, 16, 16])


In [135]:
loss=1
current_epoch_losses = {"train_loss": [], "val_loss": []}


In [147]:
import numpy as np
current_epoch_losses["train_loss"].append(loss)
current_epoch_losses["val_loss"].append(loss)  # add current iter loss to val loss list.

In [148]:
total_losses = {"train_loss": [], "val_loss": [], "curr_epoch": []}

In [149]:
for key, value in current_epoch_losses.items():
    total_losses[key].append(np.mean(value))  #
total_losses['curr_epoch'].append(1)

In [150]:
total_losses

{'train_loss': [1.0], 'val_loss': [1.0], 'curr_epoch': [1]}

In [166]:
"Train loss: {:.4f} | Validation loss: {:.4f}".format(total_losses['train_loss'][-1], total_losses['val_loss'][-1])

'Train loss: 1.0000 | Validation loss: 1.0000'